# Week 9 Problem 2

A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `YOUR CODE HERE`. Do not write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed or overwritten by the autograder.

2. Before you submit your assignment, make sure everything runs as expected. Go to menubar, select *Kernel*, and restart the kernel and run all cells (*Restart & Run all*).

3. Do not change the title (i.e. file name) of this notebook.

4. Make sure that you save your work (in the menubar, select *File* → *Save and CheckPoint*)

5. When you are ready to submit your assignment, go to *Dashboard* → *Assignments* and click the *Submit* button. Your work is not submitted until you click *Submit*.

6. You are allowed to submit an assignment multiple times, but only the most recent submission will be graded.

7. **If your code does not pass the unit tests, it will not pass the autograder.**

# Due Date: 6 PM, March 26, 2018

## Author: Kelechi Ikegwu
### Primary Reviewer: Apurv Garg

In [1]:

from nose.tools import assert_equal, assert_true, assert_false, assert_almost_equal
import numpy.testing as npt

import sklearn as sk
import numpy as np
import scipy as sp

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score

from sklearn.linear_model import LogisticRegression


The cell below will create a subdirectory under home called newsgroup_data. *If you mess up Problem 2.1 or want to the delete the newsgroup_data directory run this code in a new cell.*  
``` bash
! rm -rf /home/data_scientist/newsgroups_data
```

In [2]:
! mkdir ~/newsgroups_data
HOME = '/home/data_scientist/newsgroups_data'

### Problem 2.1
Complete the function *fetch_groups* by using the [fetch_20newsgroups](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) function to download the data. Only select the categories that are passed into the function, set the download directory to be the data_path that is passed in, and set the random_state argument to be 999 where applicable. Use the fetch_20newsgroups  function to grab the testing and training data seperately. 

*Note: Leave all of the other arguments that are not mentioned in the problem description as the default value. If you don't complete this problem properly, you may fail test throughout the rest of this notebook.*

In [5]:
def fetch_groups(_categories, data_path):
    '''
    Parameters
    ----------
    data_path: string containing path to a directory to store 20 news_group data
    _categoires: list containing categories to select from the 20 newsgroup text dataset
    
    Returns
    -------
    training data of type sklearn.datasets.base.Bunch
    testing data of type sklearn.datasets.base.Bunch
    '''
    # YOUR CODE HERE
    train = fetch_20newsgroups(data_home=data_path, subset='train', random_state=999, categories=_categories)
    test = fetch_20newsgroups(data_home=data_path, subset='test', random_state=999, categories=_categories)
    
    return train, test


In [6]:
train1, test1 = fetch_groups(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware',
      'comp.sys.mac.hardware', 'comp.windows.x'], HOME)

assert_true(isinstance(train1, sk.datasets.base.Bunch))
assert_true(isinstance(test1, sk.datasets.base.Bunch))

assert_true(set(train1.target_names) == {'comp.sys.ibm.pc.hardware', 'alt.atheism', 'comp.windows.x', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.mac.hardware'})

# testing a few slices to make sure random_state argument was set to 999
npt.assert_array_equal(train1.target[0:10], [3, 3, 4, 1, 0, 5, 0, 5, 0, 5])
npt.assert_array_equal(train1.target[150:160], [5, 2, 5, 5, 5, 1, 5, 0, 2, 3])
npt.assert_array_equal(train1.target[3140:3160], [0, 1, 1, 3, 4, 5, 2, 3, 5, 4, 5, 2, 4, 1, 0, 4, 3, 1, 5, 5])

train2, test2 = fetch_groups(['comp.os.ms-windows.misc', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos',
                              'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc'], HOME)
assert_true(isinstance(train2, sk.datasets.base.Bunch))
assert_true(isinstance(test2, sk.datasets.base.Bunch))
assert_true(set(train2.target_names) == {'talk.politics.guns', 'comp.sys.mac.hardware', 'comp.os.ms-windows.misc', 'misc.forsale', 'talk.politics.misc', 'talk.religion.misc', 'talk.politics.mideast', 'rec.autos', 'comp.windows.x'})
npt.assert_array_equal(train2.target[0:10], [6, 7, 5, 5, 0, 8, 1, 4, 4, 4])
npt.assert_array_equal(train2.target[150:160], [6, 4, 2, 6, 8, 1, 8, 0, 6, 2])
npt.assert_array_equal(train2.target[3140:3160], [3, 4, 3, 2, 8, 1, 8, 4, 2, 8, 3, 5, 5, 0, 4, 0, 6, 4, 0, 5])

### Problem 2.2
Complete the function *transformf* by learning the vocabulary and idf with a [tfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) from the training set and transforming the training and testing raw data to a term-document matrix. Use english stop words. 

*Note: Leave all of the other arguments that are not mentioned in the problem description as the default value.*

In [17]:
def transformf(train, test):
    '''
    Parameters
    ----------
    train: string containing path to a directory to store 20 news_group data
    test: list containing categories to select from the 20 newsgroup text dataset
    
    Returns
    -------
    term-document matrix containing training data (scipy.sparse.csr.csr_matrix)
    term-document matrix containing testing data (scipy.sparse.csr.csr_matrix)
    '''
    # YOUR CODE HERE
    tf = TfidfVectorizer(stop_words='english')
    tf.fit(train['data'])
    trainf = tf.fit_transform(train['data'])
    testf = tf.transform(test['data'])
    
    return trainf, testf
    

In [18]:
X_train1, X_test1 = transformf(train1, test1)
assert_true(isinstance(X_train1, sp.sparse.csr.csr_matrix))
assert_true(isinstance(X_test1, sp.sparse.csr.csr_matrix))

assert_equal(X_train1.shape, (3416, 72028))
assert_equal(X_test1.shape, (2274, 72028))

# Checking First 10 non zero elements
row1, col1 = X_train1.nonzero()
npt.assert_array_equal(row1[0:10], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
npt.assert_array_equal(col1[0:10], [49381, 16600, 28639, 58287, 22819, 24968, 60567, 50790, 57817, 20979])

assert_almost_equal(X_train1[0, 49381], 0.3571 , places=2)
assert_almost_equal(X_train1[0, 16600], 0.0997 , places=2)
assert_almost_equal(X_train1[0, 28639], 0.0919 , places=2)
assert_almost_equal(X_train1[0, 58287], 0.1729 , places=2)
assert_almost_equal(X_train1[0, 22819], 0.1065 , places=2)
assert_almost_equal(X_train1[0, 24968], 0.1238 , places=2)
assert_almost_equal(X_train1[0, 60567], 0.0132 , places=2)
assert_almost_equal(X_train1[0, 50790], 0.0644 , places=2)
assert_almost_equal(X_train1[0, 57817], 0.2152 , places=2)
assert_almost_equal(X_train1[0, 20979], 0.2614 , places=2)

X_train2, X_test2 = transformf(train2, test2)
assert_true(isinstance(X_train2, sp.sparse.csr.csr_matrix))
assert_true(isinstance(X_test2, sp.sparse.csr.csr_matrix))

assert_equal(X_train2.shape, (4893, 89431))
assert_equal(X_test2.shape, (3261, 89431))

# Checking First 10 non zero elements
row2, col2 = X_train2.nonzero()
npt.assert_array_equal(row2[0:10], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
npt.assert_array_equal(col2[0:10], [46837, 23317, 23316, 27666, 51361, 75627, 46243, 77587, 61529, 31139])

assert_almost_equal(X_train2[0, 46837], 0.324 , places=2)
assert_almost_equal(X_train2[0, 23317], 0.1644 , places=2)
assert_almost_equal(X_train2[0, 23316], 0.1644 , places=2)
assert_almost_equal(X_train2[0, 27666], 0.0506 , places=2)
assert_almost_equal(X_train2[0, 51361], 0.1644 , places=2)
assert_almost_equal(X_train2[0, 75627], 0.0137 , places=2)
assert_almost_equal(X_train2[0, 46243], 0.1701 , places=2)
assert_almost_equal(X_train2[0, 77587], 0.0727 , places=2)
assert_almost_equal(X_train2[0, 61529], 0.0143 , places=2)
assert_almost_equal(X_train2[0, 31139], 0.1041 , places=2)

### Problem 2.3

Complete the function *predictions* by using the estimator *model* from the function header. Fit the model on the training features and labels. Predict the class labels with the testing features and return the predictions.

*Note: You are not creating a new estimator. Leave all of the other arguments that are not mentioned in the problem description as the default value.*

In [39]:
def predictions(model, _X_train, _X_test, _train_data):
    '''
    Parameters
    ----------
    model: Logistic Regression sklearn estimator
    _X_train: term-document matrix containing training data (scipy.sparse.csr.csr_matrix) From Problem 2.2
    _X_test: term-document matrix containing testing data (scipy.sparse.csr.csr_matrix) From Problem 2.2
    _train_data: list containing categories to select from the 20 newsgroup text dataset From Problem 2.1
    
    Returns
    -------
    Predictions in an numpy array
    '''
    # YOUR CODE HERE
    model.fit(_X_train, _train_data.target)
    pred = model.predict(_X_test)
    
    return pred

In [40]:
pred1 = predictions(LogisticRegression(random_state=999), X_train1, X_test1, train1)
assert_true(isinstance(pred1, np.ndarray))

# Checking the predictions against the solution
npt.assert_equal(pred1, [3, 3, 5, 3, 2, 0, 4, 5, 3, 3, 1, 1, 3, 4, 2, 0, 3, 2, 3, 5, 1, 2, 2, 3, 2, 3, 3, 2, 3, 0, 5, 1, 1, 5, 4, 4, 0, 0, 1, 5, 2, 1, 3, 1, 4, 4, 0, 1, 2, 0, 2, 5, 5, 3, 3, 3, 1, 1, 2, 4, 0, 1, 3, 5, 3, 5, 4, 0, 4, 1, 4, 3, 1, 3, 5, 3, 1, 2, 4, 1, 3, 0, 4, 0, 3, 4, 0, 0, 5, 5, 3, 0, 1, 2, 4, 2, 4, 4, 1, 5, 0, 1, 3, 2, 2, 0, 4, 3, 0, 4, 2, 4, 3, 4, 1, 4, 5, 0, 3, 2, 0, 1, 4, 5, 0, 3, 4, 3, 2, 3, 1, 4, 5, 3, 4, 0, 0, 3, 0, 0, 0, 4, 1, 1, 0, 5, 5, 4, 1, 2, 4, 4, 1, 1, 2, 5, 5, 5, 0, 1, 1, 1, 3, 1, 3, 0, 0, 5, 1, 1, 1, 2, 4, 4, 2, 3, 2, 4, 5, 0, 5, 0, 4, 0, 2, 5, 5, 1, 5, 4, 4, 3, 3, 2, 3, 1, 4, 5, 2, 4, 0, 3, 4, 0, 4, 3, 3, 0, 2, 4, 0, 3, 3, 0, 4, 2, 2, 4, 3, 5, 5, 5, 4, 3, 4, 4, 2, 2, 2, 4, 1, 0, 5, 5, 2, 5, 0, 1, 5, 1, 1, 5, 0, 3, 1, 3, 0, 2, 5, 2, 4, 4, 5, 2, 5, 1, 3, 3, 0, 5, 1, 5, 2, 4, 3, 2, 0, 1, 4, 4, 5, 3, 3, 3, 1, 2, 3, 5, 1, 5, 3, 0, 5, 1, 0, 5, 3, 4, 0, 4, 3, 2, 0, 0, 1, 1, 4, 3, 5, 3, 3, 0, 0, 0, 2, 0, 3, 5, 3, 0, 3, 1, 4, 3, 0, 4, 5, 1, 2, 0, 2, 1, 5, 2, 2, 4, 0, 2, 4, 0, 4, 3, 3, 4, 5, 4, 4, 2, 5, 5, 2, 2, 3, 4, 1, 1, 2, 3, 0, 3, 0, 5, 4, 0, 4, 5, 2, 0, 1, 2, 3, 3, 3, 2, 4, 5, 3, 0, 0, 2, 2, 4, 3, 1, 1, 1, 2, 4, 4, 5, 1, 4, 5, 4, 5, 4, 2, 4, 4, 1, 4, 3, 4, 3, 2, 1, 2, 4, 1, 1, 5, 4, 3, 0, 3, 4, 5, 4, 0, 5, 2, 0, 1, 5, 4, 1, 5, 4, 5, 2, 2, 2, 1, 4, 3, 2, 3, 0, 3, 4, 1, 2, 2, 5, 4, 1, 2, 4, 3, 3, 1, 2, 5, 5, 2, 5, 5, 2, 4, 2, 3, 2, 0, 4, 1, 3, 2, 5, 1, 2, 4, 4, 3, 4, 1, 3, 0, 3, 3, 2, 4, 3, 1, 4, 5, 3, 0, 3, 5, 0, 5, 5, 5, 1, 1, 3, 5, 5, 4, 2, 2, 5, 4, 4, 1, 5, 3, 3, 1, 1, 3, 3, 2, 1, 1, 4, 5, 2, 0, 3, 0, 3, 2, 2, 1, 4, 4, 1, 0, 0, 0, 1, 5, 4, 2, 5, 0, 4, 1, 0, 0, 5, 5, 4, 5, 1, 0, 2, 5, 3, 4, 1, 2, 2, 0, 1, 0, 1, 0, 4, 1, 4, 5, 4, 0, 5, 2, 1, 2, 5, 2, 3, 0, 0, 4, 5, 3, 2, 0, 5, 3, 1, 3, 2, 3, 4, 1, 0, 2, 4, 1, 5, 2, 2, 0, 1, 5, 3, 2, 5, 1, 5, 1, 1, 5, 1, 5, 5, 1, 0, 1, 0, 4, 1, 0, 3, 3, 0, 2, 3, 5, 3, 2, 1, 2, 4, 3, 2, 5, 1, 3, 3, 0, 1, 4, 3, 5, 3, 2, 0, 2, 0, 5, 5, 3, 2, 4, 0, 5, 4, 2, 3, 4, 3, 5, 4, 1, 3, 5, 4, 4, 1, 1, 0, 1, 2, 3, 1, 0, 5, 4, 3, 2, 3, 3, 5, 1, 4, 3, 5, 1, 2, 1, 1, 4, 5, 2, 4, 4, 5, 0, 4, 2, 1, 3, 2, 3, 0, 5, 3, 1, 2, 0, 3, 5, 0, 1, 0, 2, 4, 3, 1, 2, 5, 3, 0, 4, 3, 2, 3, 2, 1, 5, 1, 4, 0, 3, 5, 0, 4, 5, 3, 0, 5, 5, 2, 1, 2, 5, 1, 3, 4, 2, 3, 5, 3, 5, 3, 5, 4, 1, 4, 5, 3, 4, 0, 4, 2, 4, 0, 4, 4, 1, 0, 1, 1, 1, 4, 4, 1, 2, 3, 3, 2, 1, 4, 4, 1, 2, 1, 1, 2, 2, 2, 4, 0, 2, 5, 0, 5, 4, 3, 0, 3, 1, 5, 2, 3, 1, 5, 4, 5, 1, 1, 1, 5, 2, 3, 5, 2, 5, 0, 0, 5, 0, 5, 5, 2, 5, 2, 5, 3, 4, 0, 1, 4, 3, 2, 1, 5, 1, 1, 4, 1, 3, 2, 2, 5, 1, 5, 1, 2, 3, 0, 4, 0, 5, 5, 1, 2, 4, 3, 3, 0, 0, 2, 0, 5, 1, 3, 0, 2, 2, 1, 4, 0, 1, 5, 4, 3, 1, 5, 1, 3, 1, 5, 5, 4, 2, 2, 1, 3, 0, 1, 4, 1, 1, 0, 5, 2, 4, 1, 0, 4, 3, 4, 5, 2, 3, 1, 5, 1, 2, 1, 2, 3, 3, 2, 2, 2, 0, 0, 3, 2, 2, 4, 3, 1, 3, 3, 4, 4, 2, 3, 0, 3, 4, 4, 2, 2, 1, 2, 2, 5, 5, 0, 4, 3, 4, 3, 4, 3, 1, 4, 1, 1, 3, 1, 0, 3, 2, 5, 5, 0, 2, 4, 2, 1, 1, 1, 1, 3, 2, 2, 4, 3, 2, 5, 5, 1, 5, 5, 2, 1, 3, 5, 0, 3, 5, 2, 3, 5, 2, 3, 0, 2, 4, 4, 0, 1, 1, 2, 2, 1, 0, 2, 3, 2, 3, 5, 4, 1, 0, 3, 0, 0, 1, 4, 4, 5, 1, 1, 4, 1, 0, 1, 2, 1, 0, 0, 2, 2, 0, 5, 0, 5, 3, 2, 4, 1, 2, 3, 3, 2, 2, 3, 1, 5, 0, 1, 1, 3, 1, 4, 3, 5, 4, 5, 4, 3, 5, 4, 4, 4, 4, 4, 5, 3, 5, 5, 3, 5, 5, 1, 5, 0, 3, 2, 2, 3, 0, 5, 5, 4, 5, 1, 1, 4, 5, 1, 2, 2, 0, 3, 0, 0, 0, 1, 0, 2, 4, 5, 1, 1, 2, 4, 0, 2, 0, 3, 2, 5, 3, 1, 3, 0, 5, 0, 4, 5, 0, 3, 3, 4, 1, 5, 2, 2, 1, 4, 2, 5, 0, 3, 0, 2, 2, 5, 2, 5, 0, 2, 4, 3, 2, 0, 3, 2, 4, 0, 4, 1, 5, 2, 3, 1, 5, 0, 4, 2, 4, 5, 3, 2, 1, 1, 0, 5, 5, 3, 1, 3, 1, 1, 4, 0, 2, 5, 3, 5, 3, 2, 5, 5, 4, 0, 1, 0, 4, 4, 3, 2, 3, 3, 4, 5, 3, 2, 2, 3, 1, 4, 0, 2, 4, 1, 0, 1, 4, 1, 5, 4, 4, 0, 2, 1, 4, 5, 4, 0, 2, 1, 0, 2, 1, 4, 2, 3, 1, 3, 1, 4, 5, 1, 3, 4, 0, 3, 3, 0, 5, 1, 0, 0, 1, 5, 1, 1, 2, 2, 2, 5, 5, 1, 4, 3, 1, 3, 1, 5, 5, 1, 4, 1, 4, 2, 1, 2, 2, 2, 2, 5, 3, 3, 2, 1, 2, 1, 5, 5, 1, 0, 2, 5, 1, 0, 4, 4, 5, 1, 4, 1, 3, 5, 4, 1, 1, 0, 1, 1, 5, 1, 2, 5, 4, 1, 4, 4, 5, 4, 2, 2, 0, 0, 3, 2, 0, 0, 3, 0, 2, 2, 4, 2, 3, 0, 1, 5, 4, 1, 2, 5, 1, 3, 5, 2, 3, 1, 4, 2, 3, 1, 1, 0, 2, 4, 1, 2, 5, 2, 3, 2, 0, 1, 5, 3, 3, 1, 1, 1, 3, 2, 0, 2, 1, 1, 0, 1, 5, 0, 2, 3, 1, 5, 2, 2, 3, 0, 2, 0, 4, 3, 3, 5, 5, 0, 4, 5, 0, 1, 5, 1, 3, 4, 2, 0, 0, 3, 2, 4, 3, 2, 4, 3, 4, 3, 3, 4, 3, 3, 3, 4, 2, 3, 3, 5, 1, 0, 3, 5, 1, 1, 1, 5, 3, 5, 4, 2, 2, 2, 0, 3, 4, 4, 5, 2, 1, 2, 3, 0, 5, 2, 1, 1, 3, 0, 4, 4, 4, 4, 0, 3, 4, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 2, 1, 4, 2, 1, 3, 2, 1, 3, 0, 2, 1, 1, 5, 5, 5, 1, 5, 0, 0, 3, 4, 5, 4, 5, 4, 1, 2, 1, 4, 1, 5, 0, 2, 5, 1, 3, 3, 0, 4, 2, 5, 5, 4, 4, 3, 1, 1, 1, 4, 1, 4, 0, 2, 4, 3, 2, 1, 5, 0, 2, 1, 0, 4, 3, 3, 5, 1, 1, 0, 2, 3, 1, 5, 4, 4, 3, 1, 5, 2, 4, 1, 2, 3, 3, 0, 5, 2, 3, 2, 2, 4, 5, 4, 2, 1, 1, 5, 1, 1, 4, 4, 4, 2, 5, 5, 0, 5, 3, 5, 1, 2, 3, 4, 4, 4, 1, 2, 5, 3, 4, 2, 3, 4, 5, 1, 4, 0, 0, 2, 4, 4, 4, 0, 1, 1, 2, 5, 4, 2, 1, 3, 3, 3, 1, 2, 5, 3, 1, 5, 0, 1, 2, 1, 1, 4, 1, 0, 1, 4, 2, 1, 4, 3, 1, 2, 3, 1, 2, 3, 5, 2, 0, 0, 5, 2, 2, 3, 1, 0, 3, 4, 0, 3, 4, 4, 4, 2, 4, 5, 0, 4, 1, 3, 1, 4, 2, 1, 0, 1, 2, 3, 4, 5, 3, 2, 0, 4, 4, 1, 3, 3, 0, 3, 3, 0, 3, 1, 4, 0, 4, 1, 0, 4, 0, 1, 0, 3, 1, 4, 3, 4, 3, 5, 3, 4, 1, 0, 3, 5, 3, 2, 5, 1, 5, 2, 5, 2, 1, 2, 1, 4, 0, 4, 2, 1, 0, 4, 4, 5, 1, 1, 4, 1, 5, 0, 2, 0, 3, 3, 0, 4, 4, 1, 4, 3, 4, 4, 3, 4, 0, 1, 4, 5, 5, 5, 0, 1, 3, 1, 1, 3, 4, 4, 1, 5, 2, 4, 3, 1, 4, 1, 1, 0, 2, 2, 4, 0, 0, 1, 3, 3, 2, 1, 5, 3, 5, 4, 2, 3, 3, 1, 1, 2, 5, 2, 3, 2, 4, 5, 1, 4, 2, 4, 2, 2, 0, 3, 5, 0, 1, 0, 2, 2, 0, 3, 2, 1, 1, 1, 3, 5, 2, 0, 5, 0, 3, 2, 2, 1, 2, 0, 0, 1, 3, 4, 5, 4, 3, 4, 4, 2, 4, 1, 1, 3, 0, 0, 1, 2, 2, 5, 2, 1, 2, 4, 3, 5, 0, 3, 2, 5, 1, 2, 4, 2, 0, 0, 5, 2, 4, 0, 2, 1, 5, 3, 5, 3, 4, 1, 1, 3, 2, 2, 0, 1, 4, 4, 5, 0, 0, 4, 4, 1, 5, 4, 4, 0, 4, 1, 4, 2, 3, 4, 0, 2, 4, 0, 0, 1, 2, 2, 0, 4, 1, 1, 0, 1, 5, 5, 1, 2, 3, 4, 3, 4, 4, 2, 0, 3, 2, 2, 3, 0, 5, 5, 4, 2, 3, 4, 2, 1, 0, 5, 4, 1, 3, 2, 3, 1, 1, 1, 1, 3, 1, 0, 1, 2, 5, 3, 0, 4, 1, 1, 1, 5, 5, 2, 4, 3, 1, 4, 2, 4, 4, 4, 2, 4, 5, 4, 0, 1, 2, 5, 2, 4, 0, 3, 1, 5, 0, 1, 3, 3, 1, 4, 1, 3, 0, 4, 2, 3, 5, 4, 3, 2, 4, 2, 0, 1, 0, 0, 1, 4, 1, 2, 1, 1, 2, 3, 1, 1, 1, 2, 2, 5, 4, 0, 3, 4, 5, 4, 3, 5, 2, 5, 2, 4, 2, 3, 0, 1, 2, 5, 2, 2, 2, 0, 0, 1, 1, 0, 2, 1, 5, 2, 1, 1, 2, 0, 2, 5, 1, 1, 1, 3, 4, 1, 0, 5, 3, 3, 4, 4, 4, 4, 1, 3, 0, 5, 2, 3, 3, 3, 5, 0, 5, 5, 1, 4, 5, 5, 1, 3, 2, 4, 4, 5, 3, 5, 0, 2, 5, 1, 2, 3, 2, 4, 2, 5, 4, 1, 3, 3, 5, 4, 3, 5, 1, 2, 0, 4, 3, 3, 5, 5, 4, 5, 4, 2, 2, 3, 1, 1, 2, 4, 0, 4, 2, 3, 4, 0, 3, 3, 4, 5, 2, 4, 1, 5, 4, 3, 4, 0, 4, 3, 3, 4, 2, 4, 3, 1, 1, 0, 2, 3, 2, 0, 2, 0, 5, 4, 1, 4, 3, 4, 1, 5, 0, 0, 1, 2, 0, 0, 2, 4, 1, 2, 4, 4, 5, 5, 0, 2, 3, 1, 2, 4, 3, 3, 4, 1, 0, 2, 0, 4, 2, 5, 1, 3, 4, 5, 2, 0, 4, 2, 1, 1, 0, 5, 3, 5, 0, 3, 2, 0, 1, 3, 4, 4, 4, 0, 5, 3, 2, 4, 2, 2, 3, 5, 5, 4, 5, 2, 5, 0, 1, 5, 1, 2, 4, 2, 3, 2, 2, 1, 3, 1, 1, 1, 5, 1, 1, 4, 2, 3, 0, 0, 3, 1, 5, 2, 5, 1, 5, 2, 2, 4, 3, 3, 0, 3, 4, 2, 2, 3, 0, 1, 4, 2, 5, 1, 5, 5, 1, 0, 2, 2, 3, 3, 0])

pred2 = predictions(LogisticRegression(random_state=999), X_train2, X_test2, train2)
assert_true(isinstance(pred2, np.ndarray))

# Checking the predictions against the solution
npt.assert_equal(pred2, [0, 1, 2, 6, 3, 0, 2, 0, 1, 1, 3, 0, 5, 2, 5, 7, 1, 1, 2, 3, 5, 3, 5, 5, 4, 4, 2, 1, 8, 5, 5, 5, 5, 4, 8, 0, 6, 0, 3, 8, 3, 7, 6, 2, 7, 6, 0, 1, 0, 7, 2, 5, 4, 4, 0, 4, 3, 7, 3, 8, 4, 7, 4, 5, 6, 7, 2, 4, 5, 2, 4, 4, 6, 0, 5, 8, 0, 3, 5, 5, 6, 5, 8, 3, 2, 6, 1, 0, 1, 3, 4, 1, 0, 0, 0, 1, 2, 3, 6, 2, 1, 1, 5, 3, 1, 3, 1, 0, 5, 3, 6, 4, 0, 4, 7, 6, 3, 8, 0, 1, 4, 4, 3, 5, 3, 0, 6, 0, 4, 5, 2, 3, 7, 4, 6, 1, 0, 0, 0, 4, 7, 4, 3, 4, 4, 3, 0, 5, 1, 8, 1, 0, 4, 3, 0, 2, 5, 3, 1, 4, 1, 5, 8, 4, 4, 0, 3, 1, 2, 1, 1, 3, 0, 4, 0, 4, 3, 3, 2, 0, 6, 1, 2, 0, 6, 3, 4, 2, 3, 0, 0, 6, 7, 5, 4, 6, 6, 7, 1, 6, 5, 0, 0, 0, 5, 1, 4, 4, 7, 5, 6, 6, 3, 1, 6, 0, 4, 1, 2, 4, 2, 0, 6, 6, 6, 5, 6, 2, 5, 0, 3, 0, 2, 6, 7, 4, 8, 1, 6, 5, 2, 1, 5, 3, 1, 4, 3, 5, 8, 0, 4, 1, 3, 5, 2, 6, 0, 5, 4, 6, 2, 2, 5, 3, 8, 7, 6, 2, 1, 5, 2, 7, 1, 5, 4, 0, 0, 3, 1, 2, 8, 7, 6, 0, 3, 8, 1, 1, 0, 6, 4, 0, 5, 7, 4, 0, 4, 6, 2, 7, 1, 5, 6, 7, 2, 0, 5, 5, 2, 0, 0, 5, 5, 7, 2, 4, 3, 3, 8, 5, 7, 5, 3, 2, 6, 3, 3, 5, 3, 0, 3, 2, 8, 8, 5, 2, 2, 1, 1, 1, 1, 6, 5, 3, 6, 4, 1, 1, 5, 0, 0, 3, 3, 3, 7, 0, 2, 2, 3, 1, 3, 8, 6, 7, 5, 2, 4, 8, 7, 5, 7, 1, 8, 0, 0, 8, 4, 0, 0, 1, 6, 7, 2, 1, 6, 3, 1, 7, 8, 3, 6, 1, 3, 1, 1, 4, 6, 2, 7, 0, 1, 6, 0, 7, 4, 0, 6, 8, 3, 5, 1, 0, 5, 4, 5, 1, 1, 3, 6, 6, 5, 3, 4, 4, 5, 6, 5, 5, 4, 0, 7, 3, 0, 4, 5, 0, 0, 7, 1, 0, 6, 7, 4, 3, 6, 5, 3, 8, 0, 5, 3, 6, 5, 4, 8, 2, 5, 4, 0, 3, 3, 1, 6, 3, 8, 7, 2, 3, 7, 1, 8, 6, 6, 8, 5, 4, 1, 3, 6, 8, 6, 5, 1, 0, 0, 1, 2, 5, 0, 2, 1, 1, 8, 3, 5, 1, 2, 2, 2, 3, 1, 5, 3, 0, 3, 5, 8, 7, 3, 7, 4, 5, 0, 4, 3, 1, 5, 2, 0, 4, 4, 3, 3, 6, 4, 8, 6, 0, 7, 5, 4, 5, 6, 4, 1, 1, 5, 5, 2, 8, 2, 2, 5, 2, 4, 6, 0, 4, 0, 6, 7, 3, 5, 6, 2, 3, 1, 1, 5, 4, 8, 5, 3, 5, 3, 8, 0, 5, 2, 2, 6, 0, 4, 4, 2, 5, 2, 3, 5, 6, 5, 5, 0, 2, 3, 4, 0, 3, 2, 5, 1, 4, 1, 1, 0, 0, 7, 3, 8, 3, 3, 1, 6, 4, 5, 6, 2, 5, 1, 5, 7, 4, 6, 4, 4, 0, 7, 3, 8, 4, 0, 4, 8, 0, 3, 2, 0, 4, 6, 2, 4, 5, 1, 7, 4, 1, 7, 6, 6, 6, 3, 7, 3, 0, 5, 4, 3, 7, 3, 1, 2, 5, 0, 4, 7, 1, 6, 0, 6, 8, 6, 5, 4, 8, 2, 5, 2, 6, 7, 6, 4, 3, 5, 0, 8, 2, 5, 1, 8, 1, 8, 2, 0, 7, 2, 1, 8, 1, 4, 7, 2, 1, 2, 1, 3, 3, 2, 4, 6, 0, 1, 3, 4, 6, 6, 2, 5, 0, 7, 2, 1, 2, 4, 7, 2, 8, 6, 0, 1, 1, 1, 0, 4, 4, 5, 5, 1, 3, 1, 8, 3, 2, 5, 6, 5, 1, 1, 3, 1, 0, 4, 5, 3, 3, 1, 0, 5, 2, 3, 3, 2, 8, 1, 1, 2, 7, 3, 1, 3, 7, 2, 0, 2, 4, 6, 8, 3, 0, 0, 0, 4, 0, 5, 6, 3, 3, 5, 5, 2, 3, 2, 4, 8, 8, 3, 2, 6, 4, 8, 2, 3, 3, 1, 5, 8, 0, 5, 5, 5, 6, 6, 6, 3, 4, 5, 8, 4, 3, 8, 4, 2, 1, 0, 1, 5, 5, 5, 5, 0, 1, 2, 7, 8, 5, 0, 3, 3, 0, 3, 4, 5, 5, 7, 7, 5, 1, 4, 2, 5, 0, 4, 8, 4, 0, 3, 5, 5, 1, 3, 7, 2, 3, 1, 1, 5, 4, 3, 6, 3, 2, 1, 0, 4, 2, 3, 3, 8, 5, 2, 7, 3, 6, 8, 1, 4, 0, 4, 0, 5, 0, 1, 1, 6, 4, 5, 4, 4, 8, 5, 5, 6, 3, 0, 6, 6, 4, 2, 4, 5, 2, 3, 3, 8, 0, 2, 3, 7, 2, 5, 6, 0, 4, 7, 3, 2, 5, 1, 0, 5, 6, 5, 1, 5, 7, 6, 2, 6, 0, 8, 1, 0, 2, 5, 2, 6, 5, 3, 3, 4, 7, 0, 5, 0, 0, 4, 8, 5, 1, 6, 0, 0, 2, 5, 1, 6, 5, 6, 6, 0, 0, 6, 0, 0, 4, 1, 1, 5, 3, 4, 6, 8, 6, 7, 1, 4, 7, 2, 0, 3, 6, 8, 6, 3, 1, 0, 6, 3, 4, 5, 5, 5, 0, 5, 5, 5, 8, 8, 6, 0, 2, 8, 5, 0, 1, 5, 7, 5, 3, 5, 8, 5, 2, 4, 0, 8, 6, 3, 3, 1, 2, 3, 2, 0, 6, 3, 2, 1, 4, 4, 5, 8, 0, 0, 4, 2, 0, 4, 0, 0, 1, 8, 6, 7, 7, 3, 4, 1, 4, 4, 2, 6, 5, 6, 5, 3, 3, 0, 4, 3, 5, 2, 3, 5, 5, 3, 2, 3, 1, 4, 7, 5, 5, 5, 2, 8, 6, 7, 1, 2, 2, 5, 4, 2, 5, 2, 0, 1, 0, 1, 6, 2, 5, 6, 0, 5, 7, 3, 3, 8, 1, 5, 2, 6, 7, 2, 7, 7, 5, 3, 3, 1, 3, 6, 0, 7, 3, 3, 2, 3, 0, 2, 0, 2, 1, 0, 2, 2, 5, 2, 6, 3, 2, 2, 0, 8, 5, 2, 8, 1, 8, 4, 4, 7, 2, 4, 5, 2, 7, 1, 3, 3, 8, 0, 0, 4, 5, 5, 4, 0, 1, 0, 7, 4, 6, 8, 4, 1, 2, 4, 8, 8, 1, 0, 7, 3, 6, 2, 4, 2, 3, 5, 5, 2, 1, 3, 0, 7, 5, 0, 4, 8, 5, 5, 3, 3, 4, 3, 2, 5, 0, 4, 5, 1, 0, 1, 2, 1, 6, 3, 4, 7, 3, 5, 0, 0, 1, 4, 5, 7, 2, 0, 8, 2, 2, 8, 3, 4, 4, 3, 5, 1, 3, 3, 5, 7, 2, 7, 7, 3, 3, 4, 0, 6, 5, 0, 3, 6, 5, 3, 2, 0, 4, 0, 4, 4, 3, 4, 0, 3, 6, 1, 3, 0, 1, 5, 5, 2, 2, 2, 0, 5, 2, 4, 2, 2, 5, 4, 4, 1, 8, 0, 4, 3, 3, 6, 5, 4, 0, 8, 5, 3, 8, 3, 2, 0, 3, 3, 2, 3, 2, 5, 5, 4, 0, 5, 8, 0, 5, 6, 7, 5, 3, 1, 6, 3, 5, 0, 0, 3, 7, 5, 2, 0, 2, 1, 4, 0, 0, 0, 6, 7, 5, 5, 3, 4, 6, 3, 2, 4, 5, 2, 5, 2, 0, 8, 3, 2, 3, 6, 6, 3, 1, 4, 8, 4, 4, 5, 4, 1, 4, 7, 1, 5, 1, 7, 2, 4, 6, 3, 6, 3, 6, 8, 4, 5, 7, 5, 0, 5, 6, 4, 4, 3, 1, 1, 3, 7, 1, 6, 5, 5, 5, 6, 4, 5, 0, 6, 1, 4, 6, 2, 1, 4, 2, 5, 3, 7, 1, 8, 2, 1, 6, 6, 5, 5, 6, 1, 2, 2, 4, 0, 5, 0, 4, 7, 0, 1, 6, 7, 2, 2, 2, 3, 5, 8, 3, 0, 3, 5, 4, 6, 0, 3, 5, 4, 2, 3, 4, 1, 5, 0, 0, 5, 5, 6, 2, 7, 2, 3, 6, 2, 0, 4, 7, 5, 2, 3, 1, 6, 4, 0, 2, 3, 0, 0, 6, 8, 4, 7, 5, 5, 5, 8, 5, 7, 1, 0, 0, 7, 2, 1, 2, 2, 5, 0, 3, 0, 5, 0, 3, 1, 2, 1, 1, 1, 2, 6, 0, 0, 0, 1, 0, 7, 1, 2, 8, 6, 1, 5, 1, 0, 6, 1, 1, 1, 1, 0, 4, 0, 4, 3, 1, 6, 8, 3, 0, 3, 6, 5, 8, 6, 6, 7, 0, 7, 8, 3, 5, 3, 7, 8, 5, 3, 0, 1, 2, 0, 5, 5, 2, 3, 2, 7, 5, 6, 4, 0, 1, 8, 6, 3, 1, 8, 2, 4, 1, 3, 2, 7, 4, 5, 1, 8, 2, 1, 6, 4, 6, 1, 1, 0, 1, 4, 2, 2, 2, 1, 5, 0, 4, 1, 5, 1, 7, 2, 4, 1, 4, 0, 1, 5, 4, 7, 8, 4, 6, 2, 4, 6, 4, 1, 6, 2, 0, 8, 5, 3, 6, 0, 2, 4, 1, 4, 6, 2, 3, 3, 1, 2, 6, 1, 0, 1, 6, 3, 1, 2, 4, 7, 3, 0, 0, 1, 4, 3, 2, 5, 2, 1, 1, 8, 4, 4, 2, 5, 8, 0, 0, 5, 6, 3, 5, 4, 4, 2, 1, 4, 0, 3, 5, 1, 1, 2, 1, 8, 8, 4, 7, 3, 5, 2, 7, 7, 4, 4, 4, 4, 1, 4, 0, 0, 6, 4, 3, 6, 3, 5, 1, 3, 0, 5, 6, 0, 5, 1, 2, 6, 1, 3, 7, 8, 0, 6, 3, 1, 1, 5, 7, 8, 5, 4, 6, 8, 1, 4, 3, 8, 1, 8, 5, 8, 5, 5, 1, 6, 5, 6, 5, 5, 3, 3, 0, 2, 3, 4, 2, 3, 4, 6, 0, 0, 5, 2, 4, 2, 5, 1, 6, 8, 7, 5, 0, 4, 7, 4, 5, 0, 4, 2, 3, 4, 0, 0, 2, 3, 2, 3, 5, 5, 8, 3, 8, 6, 2, 3, 2, 1, 1, 2, 1, 6, 4, 5, 7, 0, 0, 4, 3, 2, 5, 4, 2, 4, 1, 0, 4, 1, 2, 1, 3, 3, 0, 2, 3, 6, 3, 2, 8, 6, 4, 7, 1, 3, 0, 5, 5, 0, 1, 1, 1, 2, 5, 0, 6, 7, 1, 0, 2, 3, 2, 6, 6, 2, 6, 3, 2, 0, 0, 1, 2, 4, 2, 7, 5, 2, 6, 4, 0, 1, 4, 2, 0, 2, 1, 0, 4, 7, 0, 2, 6, 6, 5, 4, 0, 8, 3, 2, 1, 3, 5, 0, 5, 5, 2, 6, 6, 3, 3, 0, 0, 1, 2, 6, 1, 0, 3, 1, 6, 0, 4, 2, 4, 1, 5, 2, 8, 8, 0, 3, 7, 6, 3, 3, 1, 4, 5, 5, 1, 4, 6, 5, 4, 4, 1, 6, 2, 7, 6, 4, 2, 5, 7, 8, 1, 6, 5, 5, 7, 5, 7, 1, 5, 6, 7, 3, 0, 2, 6, 4, 4, 1, 4, 4, 0, 3, 5, 2, 6, 1, 8, 1, 5, 4, 4, 5, 4, 2, 6, 1, 4, 5, 1, 7, 4, 3, 4, 8, 8, 6, 7, 0, 5, 0, 1, 3, 4, 2, 3, 6, 0, 7, 0, 5, 1, 5, 3, 1, 4, 5, 4, 4, 6, 7, 5, 1, 7, 4, 1, 8, 0, 1, 0, 5, 4, 4, 4, 5, 1, 3, 4, 0, 3, 0, 4, 2, 2, 6, 4, 7, 4, 4, 5, 8, 4, 5, 6, 2, 6, 1, 4, 8, 4, 4, 7, 8, 0, 1, 3, 6, 2, 7, 2, 7, 5, 4, 7, 5, 4, 1, 0, 2, 6, 0, 2, 6, 6, 1, 4, 3, 6, 8, 1, 8, 1, 8, 5, 3, 5, 0, 0, 1, 0, 6, 4, 1, 8, 1, 6, 5, 3, 0, 3, 4, 0, 1, 6, 3, 8, 6, 4, 5, 4, 6, 3, 6, 5, 6, 1, 3, 4, 5, 2, 6, 5, 1, 1, 4, 4, 3, 2, 8, 0, 0, 5, 3, 0, 3, 1, 5, 6, 6, 3, 0, 0, 4, 7, 2, 6, 6, 7, 0, 5, 0, 8, 3, 5, 0, 7, 5, 0, 2, 2, 3, 8, 2, 6, 1, 1, 7, 4, 4, 3, 4, 0, 5, 6, 5, 3, 7, 3, 5, 3, 0, 7, 0, 4, 2, 6, 7, 7, 2, 5, 5, 1, 0, 0, 0, 0, 3, 3, 4, 3, 8, 0, 7, 0, 5, 4, 3, 2, 7, 3, 1, 4, 5, 4, 8, 2, 6, 0, 3, 7, 0, 5, 4, 8, 6, 7, 5, 2, 4, 8, 5, 8, 4, 0, 4, 2, 1, 4, 1, 0, 2, 8, 4, 6, 0, 4, 5, 0, 6, 0, 6, 6, 5, 4, 0, 4, 7, 6, 8, 4, 6, 4, 3, 7, 5, 5, 7, 6, 4, 3, 3, 5, 1, 2, 4, 4, 0, 0, 1, 6, 7, 2, 8, 5, 4, 0, 3, 4, 4, 0, 2, 1, 8, 6, 0, 8, 5, 3, 5, 3, 3, 5, 8, 4, 3, 1, 7, 8, 5, 4, 3, 0, 1, 7, 4, 2, 4, 3, 2, 5, 6, 7, 3, 0, 1, 0, 7, 5, 7, 0, 6, 8, 0, 0, 0, 4, 0, 5, 3, 0, 3, 8, 0, 6, 5, 3, 7, 1, 3, 8, 4, 6, 0, 3, 3, 3, 4, 3, 0, 1, 5, 3, 1, 1, 5, 4, 1, 3, 3, 1, 3, 0, 8, 3, 1, 6, 2, 6, 6, 1, 7, 0, 4, 4, 3, 0, 2, 2, 6, 4, 1, 3, 1, 2, 2, 1, 5, 8, 7, 2, 4, 3, 6, 7, 2, 1, 8, 5, 3, 4, 5, 0, 5, 2, 2, 5, 1, 5, 7, 5, 7, 1, 4, 5, 4, 1, 4, 8, 3, 4, 5, 3, 3, 7, 3, 5, 0, 2, 6, 1, 0, 0, 3, 3, 6, 7, 1, 5, 8, 3, 6, 3, 2, 0, 7, 7, 3, 3, 3, 0, 0, 5, 5, 6, 0, 3, 0, 6, 7, 6, 3, 5, 0, 5, 2, 3, 2, 0, 7, 8, 8, 5, 6, 2, 6, 1, 2, 2, 5, 6, 4, 7, 1, 3, 1, 3, 6, 2, 4, 2, 2, 7, 0, 2, 2, 0, 3, 0, 8, 3, 0, 8, 4, 0, 0, 5, 5, 1, 5, 7, 5, 2, 0, 8, 3, 6, 4, 5, 3, 3, 5, 1, 8, 4, 8, 0, 5, 5, 2, 8, 2, 3, 0, 6, 6, 7, 0, 0, 3, 7, 6, 4, 3, 2, 6, 0, 0, 6, 4, 1, 2, 2, 2, 8, 1, 5, 1, 2, 3, 3, 5, 6, 3, 4, 5, 8, 1, 6, 4, 7, 3, 5, 0, 4, 4, 1, 2, 4, 5, 6, 7, 7, 5, 2, 2, 5, 2, 5, 1, 2, 3, 4, 8, 6, 2, 4, 3, 4, 7, 2, 8, 6, 3, 6, 0, 6, 3, 8, 2, 8, 6, 0, 5, 5, 5, 6, 4, 6, 8, 7, 3, 6, 0, 3, 6, 6, 6, 3, 4, 5, 0, 7, 6, 0, 1, 7, 1, 1, 0, 3, 6, 6, 8, 4, 5, 1, 8, 5, 1, 6, 1, 3, 2, 0, 8, 2, 1, 4, 5, 4, 3, 3, 4, 2, 8, 5, 6, 3, 0, 8, 5, 2, 5, 7, 0, 4, 4, 7, 5, 6, 3, 0, 5, 5, 1, 2, 6, 6, 3, 8, 8, 3, 6, 7, 4, 8, 3, 0, 2, 6, 1, 5, 7, 6, 2, 5, 8, 1, 0, 4, 1, 3, 2, 8, 5, 0, 4, 4, 1, 8, 2, 4, 5, 3, 3, 8, 6, 0, 3, 3, 3, 3, 7, 0, 7, 3, 1, 5, 6, 0, 1, 1, 6, 4, 8, 8, 3, 4, 1, 7, 4, 6, 5, 3, 0, 5, 1, 2, 6, 7, 7, 1, 2, 6, 2, 5, 5, 8, 1, 1, 5, 0, 2, 3, 4, 6, 5, 7, 8, 1, 1, 6, 2, 2, 0, 3, 7, 1, 2, 6, 2, 6, 7, 0, 2, 2, 1, 7, 3, 5, 1, 1, 5, 5, 4, 4, 2, 2, 5, 1, 6, 4, 6, 0, 8, 4, 4, 2, 2, 4, 3, 2, 1, 4, 1, 1, 5, 5, 7, 1, 0, 6, 8, 4, 8, 0, 8, 5, 6, 7, 8, 6, 5, 0, 6, 0, 1, 7, 5, 1, 2, 8, 3, 0, 6, 6, 3, 3, 2, 0, 2, 1, 5, 5, 2, 2, 0, 7, 3, 1, 5, 1, 6, 1, 2, 4, 2, 1, 3, 2, 5, 4, 6, 6, 0, 7, 5, 6, 5, 0, 7, 3, 8, 1, 5, 3, 1, 4, 3, 2, 3, 4, 8, 2, 3, 6, 1, 5, 0, 5, 1, 3, 4, 2, 2, 0, 5, 2, 7, 3, 4, 1, 2, 5, 4, 5, 7, 4, 5, 4, 6, 1, 5, 8, 5, 2, 5, 7, 4, 1, 1, 3, 8, 2, 2, 5, 6, 4, 6, 5, 2, 3, 0, 4, 3, 6, 7, 0, 5, 5, 7, 6, 1, 4, 4, 1, 5, 2, 4, 4, 6, 3, 7, 7, 1, 5, 6, 4, 1, 1, 5, 6, 0, 5, 1, 4, 3, 1, 0, 8, 3, 5, 7, 8, 8, 6, 0, 3, 5, 1, 1, 2, 3, 0, 4, 5, 3, 4, 5, 0, 3, 2, 1, 1, 3, 6, 0, 4, 3, 6, 1, 5, 6, 2, 0, 7, 0, 1, 7, 0, 0, 3, 1, 5, 2, 2, 7, 4, 5, 8, 3, 4, 4, 6, 7, 3, 5, 4, 6, 5, 6, 8, 3, 1, 8, 2, 4, 4, 6, 2, 4, 1, 7, 3, 6, 8, 0, 2, 2, 0, 0, 1, 0, 7, 2, 1, 3, 3, 6, 2, 3, 6, 4, 1, 3, 5, 2, 5, 2, 2, 5, 1, 2, 5, 2, 3, 3, 5, 6, 8, 4, 5, 1, 0, 4, 1, 1, 0, 0, 3, 3, 8, 8, 7, 6, 5, 8, 4, 3, 1, 5, 0, 7, 6, 2, 2, 1, 1, 3, 7, 3, 7, 5, 2, 0, 6, 5, 2, 1, 2, 3, 0, 4, 3, 1, 5, 5, 0, 6, 2, 6, 3, 5, 1, 2, 5, 5, 4, 0, 1, 1, 4, 3, 3, 5, 6, 4, 2, 6, 4, 5, 0, 5, 6, 3, 0, 8, 1, 5, 5, 7, 6, 4, 4, 1, 1, 1, 3, 2, 2, 7, 2, 7, 4, 4, 2, 6, 0, 5, 7, 2, 5, 6, 1, 8, 6, 3, 4, 7, 1, 3, 7, 8, 5, 5, 1, 4, 1, 5, 3, 2, 0, 7, 0, 0, 6, 5, 4, 1, 6, 1, 5, 0, 1, 1, 4, 7, 6, 2, 0, 5, 2, 8, 8, 5, 3, 4, 7, 4])

Let us take a look at how well our model performs:

In [41]:
print(classification_report(test1['target'], pred1, target_names=test1['target_names']))

                          precision    recall  f1-score   support

             alt.atheism       0.96      0.95      0.96       319
           comp.graphics       0.77      0.84      0.81       389
 comp.os.ms-windows.misc       0.77      0.77      0.77       394
comp.sys.ibm.pc.hardware       0.80      0.80      0.80       392
   comp.sys.mac.hardware       0.85      0.88      0.86       385
          comp.windows.x       0.87      0.77      0.82       395

             avg / total       0.83      0.83      0.83      2274

